In [21]:
import os
import dotenv
from termcolor import colored
import numpy as np

from llm import LLM

dotenv.load_dotenv()
gpt35t = LLM("gpt-3.5-turbo")
gpt4 = LLM("gpt-4")

## 1. Make quadratic problems

In [10]:
from make_quadratic_problems import make_quadratic_problem

dotenv.load_dotenv()
llm = LLM("gpt-3.5-turbo")

for max in range(5, 50, 5):
    for factorable in [True, False]:
        filename = f'data/quadratic_problems/quadratic_problems_{max}_{factorable}.jsonl'
        if not os.path.exists(filename):
            print(f"Making problems ({max=}, {factorable=}), saving to {filename}", 'blue')
            with open(f'data/quadratic_problems/quadratic_problems_{max}_{factorable}.jsonl', 'w') as outfile:
                for _ in range(100):
                    outfile.write(make_quadratic_problem(max, factorable) + '\n')
        else:
            print(colored(f"Skipping making problems ({max=}, {factorable=}) because {filename} already exists", 'blue'))

Skipping making problems (max=5, factorable=True) because data/quadratic_problems/quadratic_problems_5_True.jsonl already exists
Skipping making problems (max=5, factorable=False) because data/quadratic_problems/quadratic_problems_5_False.jsonl already exists
Skipping making problems (max=10, factorable=True) because data/quadratic_problems/quadratic_problems_10_True.jsonl already exists
Skipping making problems (max=10, factorable=False) because data/quadratic_problems/quadratic_problems_10_False.jsonl already exists
Skipping making problems (max=15, factorable=True) because data/quadratic_problems/quadratic_problems_15_True.jsonl already exists
Skipping making problems (max=15, factorable=False) because data/quadratic_problems/quadratic_problems_15_False.jsonl already exists
Skipping making problems (max=20, factorable=True) because data/quadratic_problems/quadratic_problems_20_True.jsonl already exists
Skipping making problems (max=20, factorable=False) because data/quadratic_proble

## 2. Make quadratic-solving contexts with GPT-3.5-turbo and GPT-4

In [27]:
from make_quadratic_contexts import solve_quadratic_problems

# prompt is an unprincipled DoF here.
# pros: makes expected switching behavior clear
# cons: model is still in "assisstant mode", not "trying to solve the problem as efficiently as possible mode"
prompt = "Please find the roots of the quadratic equation {equation}. Start by trying to factor the equation. If you can't factor it, then use the quadratic formula. If you factor the equation successfully, do not use the quadratic formula."

# remember to force feed the model so it starts by attempting factoring! Otherwise switch rate will be too low.
false_start = "First, I'll try solving this equation by factoring."

for llm in [gpt35t, gpt4]:
    for problem_filename in os.listdir('data/quadratic_problems'):
        outfile = f'data/quadratic_contexts_{llm.model_name}/' + str.replace(problem_filename, 'problem', 'context')
        if not os.path.exists(outfile):
            print(f"Solving problems in {problem_filename}, writing to {outfile}")
            solve_quadratic_problems(problem_filename, prompt, false_start, llm)
        else:
            print(colored(f"Skipping solving problems in {problem_filename} because {outfile} already exists", 'blue'))

Solving problems in quadratic_problems_10_True.jsonl, writing to data/quadratic_contexts_gpt-3.5-turbo/quadratic_contexts_10_True.jsonl


100%|██████████| 100/100 [00:33<00:00,  2.95it/s]


## 3. Split contexts into prefixes and assess switching

In [24]:
for context_filename in os.listdir('data/quadratic_contexts_gpt-3.5-turbo'):
    with open(f'data/quadratic_contexts_gpt-3.5-turbo/{context_filename}', 'r') as file:
        print(context_filename, np.mean([len(x) for x in file.readlines()]))

quadratic_contexts_35_True.jsonl 577.3
quadratic_contexts_30_True.jsonl 559.38
quadratic_contexts_35_False.jsonl 650.49
quadratic_contexts_45_False.jsonl 671.45
quadratic_contexts_20_False.jsonl 635.85
quadratic_contexts_25_False.jsonl 621.8
quadratic_contexts_40_False.jsonl 661.83
quadratic_contexts_30_False.jsonl 635.73
quadratic_contexts_5_True.jsonl 487.8
quadratic_contexts_20_True.jsonl 519.13
quadratic_contexts_10_False.jsonl 639.27
quadratic_contexts_15_False.jsonl 604.58
quadratic_contexts_25_True.jsonl 541.18
quadratic_contexts_15_True.jsonl 516.14
quadratic_contexts_40_True.jsonl 566.6
quadratic_contexts_10_True.jsonl nan
quadratic_contexts_5_False.jsonl 632.04
quadratic_contexts_45_True.jsonl 564.0


## 4. Assess 1t and COT CPC for prefixes

## 5. Save results and analyze